In [2]:
%matplotlib inline

In [3]:
#export
from exp.nb_06 import *
from torch import nn

In [20]:
xTraining, yTraining, xValidation, yValidation = list(map(lambda x: x.cuda(),getMnistData()))
xTrainingNormalized, xValidationNormalized = normalizeVectors(xTraining, xValidation)
(xTrainingNormalized.mean(), xTrainingNormalized.std, xValidationNormalized.mean(), xValidationNormalized.std())

(tensor(1.6044e-08, device='cuda:0'),
 <function Tensor.std>,
 tensor(2.1759e-08, device='cuda:0'),
 tensor(1., device='cuda:0'))

In [21]:
numberOfClasses = 10
hiddenLayerOutput = 50
batchSize = 64
lossFuction = Functional.cross_entropy

In [22]:
trainingDataSet, validationDataSet = Dataset(xTrainingNormalized, yTraining), Dataset(xValidationNormalized, yValidation)

In [23]:
trainingDataLoader, validationDataLoader = createDataLoaders(trainingDataSet, validationDataSet, batchSize)

In [24]:
imageDataBunch = DataBunch(trainingDataLoader, validationDataLoader, numberOfClasses)

In [25]:
layerSizes = [8, 16, 32, 64, 64]

In [27]:
phases = [0.3, 0.7]
weightsScheduler = aggregateSchedulers(phases, createCosineSchedulers(0.3, 0.6, 0.2)) 
biasScheduler = aggregateSchedulers(phases, createCosineSchedulers(0.9, 1.8, 0.6))

In [38]:
class Subscriber:
    def __init__(self):
        self._totalEpochs = 0
        self._currentEpoch = 0

    def postEpoch(self, epochNumber):
        pass

    def postBatchEvaluation(self, loss):
        pass

    def preBatchEvaluation(self):
        pass

    def preEpoch(self, epoch):
        self._currentEpoch = epoch
        pass

    def preModelTeach(self, model, epochs):
        self._totalEpochs = epochs
        pass

    def postModelTeach(self):
        pass


In [39]:
class ValidationSubscriber(Subscriber):

    def __init__(self):
        pass

    def postEpoch(self, epochNumber):
        pass


In [40]:
class TrainingSubscriber(Subscriber):

    def __init__(self,
                 schedulingFunctions=[cosineScheduler(1e-1, 1e-6), cosineScheduler(1e-1, 1e-6)]):
        super().__init__()
        self._optimizer = None
        self._schedulingFunctions = schedulingFunctions

    def postEpoch(self, epochNumber):
        pass

    def preModelTeach(self, model, epochs):
        super().preModelTeach(model, epochs)
        self._optimizer = optim.SGD(model.parameters(), self._schedulingFunctions[0](0))
        self._totalEpochs = epochs

    def postBatchEvaluation(self, loss):
        loss.backward()
        self._optimizer.step()
        self._optimizer.zero_grad()

    def preBatchEvaluation(self):
        for parameterGroup, schedulingFunction in zip(self._optimizer.param_groups, self._schedulingFunctions):
            parameterGroup['lr'] = schedulingFunction(self._currentEpoch / self._totalEpochs)


In [31]:
class TeacherEnhanced:
    def __init__(self,
                 dataBunch,
                 lossFunction,
                 trainingSubscriber: TrainingSubscriber,
                 validationSubscriber: ValidationSubscriber):
        self._dataBunch = dataBunch
        self._lossFunction = lossFunction
        self._trainingSubscriber = trainingSubscriber
        self._validationSubscriber = validationSubscriber

    def teachModel(self, model, numberOfEpochs):
        self._notifiyPreTeach(model, numberOfEpochs)
        for epoch in range(numberOfEpochs):
            self._trainModel(model,
                             epoch)
            self._validateModel(model,
                                epoch)
        self._notifiyPostTaught()

    def _notifiyPreTeach(self, model, epochs):
        self._trainingSubscriber.preModelTeach(model, epochs)
        self._validationSubscriber.preModelTeach(model, epochs)

    def _notifiyPostTaught(self):
        self._trainingSubscriber.postModelTeach()
        self._validationSubscriber.postModelTeach()

    def _trainModel(self, model, epoch):
        self._processData(model,
                          self._dataBunch.trainingDataSet,
                          epoch,
                          self._trainingSubscriber)

    def _validateModel(self, model, epoch):
        with torch.no_grad():
            self._processData(model,
                              self._dataBunch.validationDataSet,
                              epoch,
                              self._validationSubscriber)

    def _processData(self,
                     model,
                     dataLoader,
                     epoch,
                     dataProcessingSubscriber: Subscriber):
        dataProcessingSubscriber.preEpoch(epoch)
        for _xDataBatch, _yDataBatch in dataLoader:
            dataProcessingSubscriber.preBatchEvaluation()
            _predictions = model(_xDataBatch)
            calculatedLoss = self._lossFunction(_predictions, _yDataBatch)
            dataProcessingSubscriber.postBatchEvaluation(calculatedLoss)
        dataProcessingSubscriber.postEpoch(epoch)


In [41]:
validationSubscriber = ValidationSubscriber()

In [42]:
trainingSubscriber = TrainingSubscriber()

In [43]:
teacher = TeacherEnhanced(imageDataBunch, 
                          lossFuction, 
                          trainingSubscriber,
                          validationSubscriber
                         )

In [45]:
convolutionalModelSR1 = createBetterConvolutionModel(numberOfClasses, layerSizes).cuda()

In [46]:
accuracy(convolutionalModelSR1(xValidationNormalized), yValidation)

tensor(0.0983, device='cuda:0')

In [47]:
teacher.teachModel(convolutionalModelSR1, 2)

In [48]:
accuracy(convolutionalModelSR1(xValidationNormalized), yValidation)

tensor(0.9749, device='cuda:0')

Todo: Statistics Subscriber, Hooks